# Adam Thomson - PHY 573 - Genomic Grover Search

The goal of this project is to demonstrate the ability for Grover's algorithm to perform simple bioinformatic queries. I will do this by first examining why genomic sequence searching is an appropriate application of Grover's algorithm, and a thorough instruction for a trivial example. I will then focus on how to construct the oracle gate for a given genomic query, as well as the full quantum circuit to implement the algorithm. I finish the demonstration by comparing the results from more complex queries when running on a local simulator vs. real hardware. The demonstration will conclude with thoughts about the current limitations of this approach and what the future may hold for the potential field of quantum bioinformatics!

In [9]:
# Import libraries
from IPython.display import Math, HTML
import math
import matplotlib

# Imports from Qiskit
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import GroverOperator, MCMT, ZGate
from qiskit.visualization import plot_distribution, plot_histogram

# Imports from Qiskit Runtime for running on real hardware
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Declare shortcut function
def dm(x):
    return display(Math(x))

## Grover's Algorithm

Grover's algorithm, also known as the quantum search algorithm, was originally published as a way to search an “unsorted database…of which just one item satisfies a given condition,”$^{1}$ proving itself to be "significantly faster than any classical algorithm can be."$^{1}$ Since then, Grover and others have developed enhancements to the algorithm;$^{2}$ for this notebook I will leverage the ability to search for potentially multiple items satisfying the condition. Vazirani provides a timeless comparison for what this algorithm does,$^{3}$ trying to find a needle in a haystack. A classical computer's approach is to scan every object in the stack until coming across the needle, which may require checking the entire haystack. But Grover's quantum approach is like "using a giant magnet so...you find your needle very quickly."$^{3}$ On average, the classical approach requires (N/2) steps to find the solution, while the quantum algorithm will find it in √N steps!

The quantum search algorithm has the potential for profound implications in many fields of research, such as biophysics and chemistry, leading to advancements in medical devices and pharmaceuticals. Shortly after Grover published his algorithm, Hollenberg showed how it could be applied to "protein sequence analysis"$^{4}$ and envisioned the same approach scaled up for "nucleotide sequence comparison in DNA."$^{4}$ In this notebook, I will demonstrate exactly that.

However, let's first examine exactly how the algorithm performs so much better than the classical counterpart.

In [96]:
# Describe the problem statement
dm(r"\text{Say you have a function } U_\omega \text{ such that }")
dm(r"""U_\omega(x) = \begin{cases}
0 &\text{for } x \neq \omega \\
1 &\text{for } x = \omega
\end{cases}""")
dm(r"\text{Where } 0 \leq x \leq (N-1), N = 2^n")
dm(r"\text{We want to find } \omega")

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The quantum approach has 3 components: setup, the oracle function, and the diffusion operator.
The initial setup only happens once, and then the oracle and diffusion operator are repeated a number of times dependent on the problem.
Construction of the oracle function is unimportant for the sake of the algorithm, only that it phase flips the correct qubits.
The diffusion operator will then perform 'inversion about the mean' to amplify the desired state(s) while dampening all others.
Finally, after enough iterations, we measure the qubits and check the answer to verify the result.

In [43]:
# State initialization
dm(r"\text{The algorithm setup is one that many quantum algorithms use, putting all input qubits into a uniform superposition.}")
dm(r"\text{This is achieved by applying a Hadamard gate to all qubits.}")
dm(r"\ket{\psi_0} = \ket{00..0}")
dm(r"\ket{\psi_0'} = H^{\otimes n} \ket{\psi_0} = (H\ket0) \otimes (H\ket0) \otimes ... (H\ket0)")
dm(r"\qquad = \frac1{\sqrt2}(\ket{0} + \ket{1}) \otimes \frac1{\sqrt2}(\ket{0} + \ket{1}) \otimes ... \frac1{\sqrt2}(\ket{0} + \ket{1})")
dm(r"\qquad = \frac1{2^{\frac{n}{2}}} (\ket{0} + \ket{1})^{\otimes n}")
dm(r"\ket{\psi_0'} = \frac1{\sqrt{N}} \sum_{x=0}^{N-1} \ket{x}")

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [49]:
# Apply the oracle operator
dm(r"\text{The oracle operator will then perform a phase flip on the qubits corresponding to a solution state.}")
dm(r"\ket{\psi_0''} = U_\omega \ket{\psi_0'}")
dm(r"""\qquad = \frac1{\sqrt{N}} \sum_{x=0}^{N-1} \begin{cases}
\ket{x} &\text{for } x \neq \omega \\
-\ket{x} &\text{for } x = \omega
\end{cases}""")

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [70]:
# Apply the diffusion operator
dm(r"\text{And the diffusion operator will amplify the state } \ket{\omega} \text{ as well as dampen all other } \ket{x}")
dm(r"U_s = 2\ket{s} \bra{s} - 1 \text{; Where } \ket{s} = H^{\otimes n}\ket{0}_n")
dm(r"\ket{\psi_1} = U_s \ket{\psi_0''} = U_s U_\omega H^{\otimes n} \ket{\psi_0}")
dm(r"\text{Depending on the size of the search space and the number of solutions, now may be a good time to measure. But if not, repeat...}")
dm(r"\ket{\psi_2} = U_s U_\omega \ket{\psi_1} = (U_s U_\omega)^2 H^{\otimes n} \ket{\psi_0}")
dm(r"\ket{\psi_t} = (U_s U_\omega)^t H^{\otimes n} \ket{\psi_0}")

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [62]:
# Describe how to determine optimal t
dm(r"\text{There are multiple ways to approach a calculation for an optimal } t, \text{ with the following equation widely used:}")
dm(r"t = \lfloor \frac{\pi}4 \sqrt{\frac{N}{m}} \rfloor")
dm(r"\text{Where } N \text{ is the size of the search space, while } m \text{ is the number of matches.}")
dm(r"\text{I will demonstrate what happens when you use a non-optimal } t \text{ later in the notebook.}")

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Let's examine in detail how these pieces come together for a simple genomic sequence search.

## Example Walkthrough

I'll start by examining each intermediary state in the trivial example of a 4-basepair reference sequence and 1-basepair search string with a single match. In this case, we are attempting to find the index of "A" in the sequence `GCAT`

In [7]:
# We will need 2 qubits to capture all 4 possible indexes for a match
dm(r"\ket{\psi_0} = \ket{00}")

<IPython.core.display.Math object>

In [64]:
# Begin by putting all qubits into a superposition with Hadamard gates
dm(r"\ket{\psi_0'} = (\Eta \otimes \Eta)\ket{\psi_0}")
dm(r"\qquad = (H \otimes H)\ket{00}")
dm(r"\qquad = H \ket0 \otimes H \ket0")
dm(r"\qquad = \frac1{\sqrt2}(\ket0 + \ket1) \otimes \frac1{\sqrt2}(\ket0 + \ket1)")
dm(r"\qquad = \frac12(\ket{00} + \ket{01} + \ket{10} + \ket{11}")


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [73]:
# Since "A" is in the 3rd position of GCAT, we know the oracle performs a phase flip for |10>
dm(r"\ket{\psi_0''} = U_\omega \ket{\psi_0'}")
dm(r"\qquad = \frac12(\ket{00} + \ket{01} - \ket{10} + \ket{11})")


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [94]:
# Apply the diffusion operator
dm(r"\ket{\psi_1} = U_s \ket{\psi_0''} = (2\ket{s} \bra{s} - 1)\ket{\psi_0''}")
dm(r"\qquad = 2\ket{s}\braket{s|\psi_0''} - \ket{\psi_0''}")
dm(r"\braket{s|\psi_0''} = \frac1{2^2}(1 + 1 - 1 + 1) = \frac12")
dm(r"\ket{\psi_1} = \ket{s} - \ket{\psi_0''}")
dm(r"""\qquad = \frac12(\cancel{\ket{00}} + \cancel{\ket{01}} + \ket{10} + \cancel{\ket{11}})
    - \frac12(\cancel{\ket{00}} + \cancel{\ket{01}} - \ket{10} + \cancel{\ket{11}})""")
dm(r"\ket{\psi_1} = \ket{10}")

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

With just a single iteration of Grover's algorithm, we have (theoretically) put the qubits into a state with 100% probability of measuring the correct answer |10> = "A"! Recall that A classical computer would take 2.5 attempts on average, with a worst case maximum of 4.

### Utility functions

Before continuing to building circuits, I'll describe some utility functions for the genomic sequence application.

In [105]:
"""
Search the input reference string for instances of the search string(s)
and return indexes for the start of all matches.
"""
def _find_match_indexes(search_seqs, reference, num_qubits):
    # If only searching for a single sequence, convert to list
    if not isinstance(search_seqs, list):
        search_seqs = [search_seqs]
   
    # Initialize list of indexes to be returned
    all_marked_indexes = []
    
    # Loop through all searches, and find all matches for each
    for search_seq in search_seqs:
        # Reset search start index to beginning of reference
        i = 0
        # Continue finding matches until we reach the end of reference
        while i < len(reference):
            try:
                n = reference.index(search_seq, i)
                all_marked_indexes.append(int(n / 2))
                # Next search will only check the reference after the current match
                i = n + 1
            # Will always eventually hit this when no matches exist in the rest of reference
            except ValueError:
                i = len(reference) + 1
    
    # Convert indexes to 0-padded binary strings
    return [bin(i)[2:].zfill(num_qubits) for i in all_marked_indexes]

In [103]:
"""
Build a Grover Operator gate, consisting of both the oracle function
and diffusion operator for a given list of search strings and reference

Parameters:
    base_seqs <str|list>: Sequence(s) being searched for
    reference <str>: Reference being searched

Returns:
    QuantumCircuit: Quantum circuit representing Grover Operator gate
"""
def genomic_oracle_gate(base_seqs, reference):
    # Compute the number of qubits required for circuit, log_2(N) (always round up)
    num_qubits = math.ceil(math.log(len(reference), 2))

    # Initialize quantum circuit with that many qubits
    qc = QuantumCircuit(num_qubits)
    
    # Search the reference for the desired strings and mark the indexes found
    marked_states = _find_match_indexes(base_seqs, reference, num_qubits)
    # Print it out for manual verification
    print(f"Marked the indexes: {marked_states}")
    
    qc = QuantumCircuit(num_qubits)
    # Mark each target state in the input list
    for target in marked_states:
        # Flip target bit-string to match Qiskit bit-ordering
        rev_target = target[::-1]
        # Find the indices of all the '0' elements in bit-string
        zero_inds = [ind for ind in range(num_qubits) if rev_target.startswith("0", ind)]
        # Add a multi-controlled Z-gate with pre- and post-applied X-gates (open-controls)
        # where the target bit-string has a '0' entry
        qc.x(zero_inds)
        qc.compose(MCMT(ZGate(), num_qubits - 1, 1), inplace=True)
        qc.x(zero_inds)

    # Also return the number of marked states found for determining number of iterations
    return qc, len(marked_states)

In [104]:
"""Determine the optimal number of iterations"""
def optimal_num_iterations(match_count, num_qubits):
    # t = floor((pi/4)*(sqrt(N/m))), at least 1
    return max(
        math.floor(
            (math.pi / 4) * (math.sqrt(match_count / 2**num_qubits))
        ), 1)

In [ ]:
"""Given a reference and input sequence, build a circuit to find indexes of matches"""
def build_genomic_grover(search_seqs, ref_str):
    # Figure out how many qubits are needed for the circuit
    nqubits = math.ceil(math.log(len(ref_str), 2))
    qc = QuantumCircuit(nqubits)

    # Initialize by applying Hadamard gates to all qubits
    qc.h(range(nqubits))

    # Build the oracle function
    

## Example Circuit

In [101]:
# Import test references
from constants import REFERENCE_GENOMES

In [ ]:
# Initialize local simulator
from qiskit_aer import AerSimulator

sim_sampler = AerSimulator()

In [ ]:
# Setup trivial example
trivial_ref = "GCAT"
trivial_search = "A"

# Now convert each to their respective bstrs
trivial_ref_bstr = _convert_bp_seq_to_bstr(trivial_ref)
trivial_search_bstr = _convert_bp_seq_to_bstr(trivial_search)

## References

Algorithm implementation based on qiskit tutorial notebook: https://learning.quantum.ibm.com/tutorial/grovers-algorithm 

[1] Grover, L. “A fast quantum mechanical algorithm for database search” Proceedings, 28th Annual ACM Symposium on the Theory of Computing May, 1996 https://doi.org/10.48550/arXiv.quant-ph/9605043

[2] Grover, L., Rudolf, T. “Creating superpositions that correspond to efficiently integrable probability
distributions” 2002 https://doi.org/10.48550/arXiv.2406.13785 

[3] Vazirani, U. "Lecture 11 1 NEEDLE IN A HAYSTACK" 2019, Sandro Mareco, https://youtu.be/-nFbjqYyoEI?list=PLXEJgM3ycgQW5ysL69uaEdPoof4it6seB

[4] Hollenberg, L. “Fast Quantum Search Algorithms in Protein Sequence Comparison - Quantum Biocomputing” 25, Feb 2000 https://doi.org/10.48550/arXiv.quant-ph/0002076 